In [8]:
pip install schedule

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep
import random
import re
import pandas as pd
from selenium import webdriver
import sys
import time
import schedule
import threading

In [2]:
from sqlalchemy import create_engine, Date, String
from sqlalchemy.dialects.oracle import NUMBER, VARCHAR2

In [3]:
# 데이터 베이스 연결 정보 설정
db_username = 'base_man'
db_password = '1111'
db_host = '192.168.0.29' # 다른 IP 주소
db_port = '1521' #Oracle의 기본포트
service_name = 'XE'

engine = create_engine(f'oracle+cx_oracle://{db_username}:{db_password}@{db_host}:{db_port}/?service_name={service_name}')

In [4]:
query = "select * from match_schedule where match_date = TRUNC(SYSDATE)"
mf = pd.read_sql(query, con=engine)

In [5]:
mf

,match_id,match_date,match_time,status,team1_code,team2_code,home_team_code
0,10,2024-09-05,18:30,예정,HH,HT,HT
1,11,2024-09-05,18:30,예정,KT,LT,LT
2,12,2024-09-05,18:30,예정,SK,LG,LG
3,13,2024-09-05,18:30,예정,WO,NC,NC


In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# 크롬 드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')

# 스케줄 중단 플래그
stop_event = threading.Event()

def collect_vote():
    print("데이터 수집 시작...")
    start_time = time.time()
    # 드라이버 초기화
    driver = webdriver.Chrome(options=options)
    
    # 결과를 저장할 리스트
    results = []
    
    # 데이터 크롤링
    for _, row in mf.iterrows():
    #    match_date = row['match_date'].replace('-', '')  # '2024-09-04' -> '20240904'
        match_date = pd.to_datetime(row['match_date']).strftime('%Y%m%d')
        h_club_code = row['team1_code']
        k_club_code = row['team2_code']
        
        # URL 생성
        url_link = f"https://m.sports.naver.com/game/{match_date}{h_club_code}{k_club_code}02024/preview"
    
        # URL로 이동
        driver.get(url_link)
        
        # 페이지 로드 대기
        time.sleep(3)
        
        try:
            # 응원수 추출
            v_votes = driver.find_element(By.XPATH, '//*[@id="content"]/div/aside/div/div[2]/div[1]/div[2]/div[1]/div[1]/div/div[2]').text
            s_votes = driver.find_element(By.XPATH, '//*[@id="content"]/div/aside/div/div[2]/div[1]/div[2]/div[2]/div[2]/div/div[2]').text
    
            # 구단 이름 추출
            v_club = driver.find_element(By.XPATH, '//*[@id="content"]/div/aside/div/div[1]/div/div[2]/div[1]/div[1]/div/div[2]/div').text
            s_club = driver.find_element(By.XPATH, '//*[@id="content"]/div/aside/div/div[1]/div/div[2]/div[1]/div[2]/div/div[2]/div').text
    
             # 데이터 변환
            club_mapping = {'한화': 'HH', 'KIA': 'HT','롯데':'LT','두산':'OB','삼성':'SS','SSG':'SK','키움':'WO'}
            
            # 데이터 저장
            results.append({
                'VSDate': row['match_date'],
                'Team1_Code': club_mapping.get(v_club, v_club),
                'Team2_Code': club_mapping.get(s_club,s_club),
                'Team1_Votes': v_votes,
                'Team2_Votes': s_votes,
            })
    
        except Exception as e:
            print(f"An error occurred for URL {url_link}: {e}")

    # 데이터프레임 생성
    result_df = pd.DataFrame(results)
    
  
    result_df.to_sql(
        'vs_vote', 
        con=engine, 
        if_exists='replace', 
        index=False, 
        dtype={
            'VSDate' : Date,
            'team1_code': VARCHAR2(10),
            'team2_code': VARCHAR2(10),
            'team1_votes': VARCHAR2(20), 
            'team2_votes': VARCHAR2(20)
        }
    )
    print("Table insert successfully")
    
    # 브라우저 종료
    driver.quit()
    end_time = time.time()  # 종료 시간 측정
    print(f"작업 소요 시간: {end_time - start_time:.2f} 초")

# 1분마다 collect_vote 함수를 실행
schedule.every(1).minutes.do(collect_vote)

# 스케쥴러 실행
while True:
    schedule.run_pending()
    time.sleep(1)

데이터 수집 시작...
Table insert successfully


NameError: name 'start_time' is not defined